# Basic Agentic RAG

The Router leverages three CSV files to understand the user's financial data:

| Data Source | Description |
|---|---|
| Income.csv | Records income streams and their values (e.g., salary, investments, rental income). |
| Expenses.csv | Tracks monthly expenditures categorized by source (e.g., rent, utilities, groceries). |
| Savings.csv | Stores data on different savings sources and their monthly amounts (e.g., emergency fund, retirement savings). |

**Query Engines:**

The Router interacts with two primary query engines:

* **Vector Search and Retrieval:** This engine retrieves documents relevant to the user's query by analyzing the semantic similarity between the query and documents in the index. This is ideal for finding specific information within the data.
* **Summarization:** This engine provides a concise summary of all documents within the index. This is useful for getting a high-level overview of the data.

**Query Processing:**

The Router analyzes the user's query and the data available in the CSV files to determine the most suitable query engine:

* If the query seeks specific details within the data (e.g., "What was my total income from freelancing last month?"), the router routes the query to the vector search and retrieval engine.
* If the query requires a general overview of the data (e.g., "Provide a summary of my income, expenses, and savings for the past year"), the Router directs the query to the summarization engine.

This approach ensures that the user receives the most relevant and efficient execution based on their intended purpose.

**Next Steps:**

In the following sections, we will delve deeper into the implementation details of the Router agent and how it interacts with the query engines and data sources.

## Setup

In [3]:
import os
from dotenv import load_dotenv, find_dotenv

In [4]:
_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [6]:
import nest_asyncio
nest_asyncio.apply()

## Load Data

_Steps before loading the documents using CSV reader_ 

- Create a folder called "data" in the current directory.
- Upload your expenses.csv, income.csv and savings.csv

In [7]:
# Load documents
from llama_index.core import SimpleDirectoryReader
from llama_index.readers.file import CSVReader

In [8]:
# CSV Reader example
parser = CSVReader()
file_extractor = {".csv": parser} 

documents = SimpleDirectoryReader(
            "./data", file_extractor=file_extractor
                ).load_data()

In [9]:
# check how many files were read
len(documents)

3

In [10]:
# Check all file names
file_names = [doc.metadata['filename'] for doc in documents ]

In [11]:
file_names

['Expenses.csv', 'Income.csv', 'Savings.csv']

In [12]:
# check contents ofa document
documents[2]

Document(id_='e31eb70d-549a-45d2-8683-2d44ca3e5f0f', embedding=None, metadata={'filename': 'Savings.csv', 'extension': '.csv', 'file_path': '/Users/bhartisinha/Downloads/RAGAgentic/data/Savings.csv', 'file_name': 'Savings.csv', 'file_type': 'text/csv', 'file_size': 580, 'creation_date': '2024-05-26', 'last_modified_date': '2024-05-26'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Savings Type, Month, Saved by, Amount, Description\nSaved, March, Adam, 1905, 10% of salary\nSaved, March, Eve, 2172, 10% of Salary\nSaved, March, Adam, 1956, Common Savings\nSaved, March, Eve, 816, Common Savings\nSaved, April, Adam, 1185, 10% of Salary\nSaved, April, Eve, 1324, 10% of Salary\nSaved, April, Adam, 2032, Porche Macan\nSaved, April, Eve, 1797, Porche Macan\nSa

In [13]:
# perform some general settings, we can change models and embeddings to be used for the project
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# Settings.llm = OpenAI(model="gpt-3.5-turbo", api_key=OPENAI_API_KEY)
Settings.llm = OpenAI(model="gpt-4", api_key=OPENAI_API_KEY)

Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002", api_key=OPENAI_API_KEY)

## Create `Summary Index` and `Vector Index` over our data

- Once the documents are read, we need to split the documents into nodes. 

- Each node contains a chunk of the documents with a unique ID, embeddings, and metadata with information on the file name, format, path and date_modified, data_created and the content within that chunk. 


In [14]:
from llama_index.core.node_parser import SentenceSplitter

# The token chunk size for each chunk is 1024
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [15]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

In [16]:
# Get metadata information for all the documents in the index
summary_index.ref_doc_info

{'bcb3a837-327e-4b59-ada9-6809ef73c725': RefDocInfo(node_ids=['141b9dc5-3028-4ddc-96a5-60f4f6463ca7'], metadata={'filename': 'Expenses.csv', 'extension': '.csv', 'file_path': '/Users/bhartisinha/Downloads/RAGAgentic/data/Expenses.csv', 'file_name': 'Expenses.csv', 'file_type': 'text/csv', 'file_size': 1640, 'creation_date': '2024-05-26', 'last_modified_date': '2024-05-26'}),
 'fb488373-bd5b-4413-b43a-28219571e5b4': RefDocInfo(node_ids=['c9257766-f2b5-4392-a62c-68d9027fde51'], metadata={'filename': 'Income.csv', 'extension': '.csv', 'file_path': '/Users/bhartisinha/Downloads/RAGAgentic/data/Income.csv', 'file_name': 'Income.csv', 'file_type': 'text/csv', 'file_size': 361, 'creation_date': '2024-05-26', 'last_modified_date': '2024-05-26'}),
 'e31eb70d-549a-45d2-8683-2d44ca3e5f0f': RefDocInfo(node_ids=['a9ac8b2e-79ac-4e95-a052-4faed5dadf5f'], metadata={'filename': 'Savings.csv', 'extension': '.csv', 'file_path': '/Users/bhartisinha/Downloads/RAGAgentic/data/Savings.csv', 'file_name': 'Sav

## Define Query Engines and Query Tools

**Vector query engine**: Querying a vector index returns similar documents based on embedding similarity. 

**Summary query engine**: Querying a summary index returns all nodes in the index and is independent of the query. 

These indexes are turned into Query engines and then into tools. Tools are query engines with a description of the tools' purpose. 

In [17]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [18]:
from llama_index.core.tools import QueryEngineTool

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to incomes, savings and expenses"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the incomes, savings and expenses tables."
    ),
)

- There are two ways to create tools, one is using **QueryEngineTool** from the tools module. This takes in the query engine and its description to transform it into a tool.

- Another is **FunctionTool** from the tools module too, which takes any Python function and converts it to a tool to be used by an agent. 

## Define Router Query Engine

- Given a query, a router picks which tool is best for answering that query and then implements that engine. 

- The router has dynamic query understanding capabilities. 

- We can choose different selectors to build a router. These selectors have unique attributes and can perform a task in different ways.

- The one used here is LLMSingleSelector; which prompts the LLM to output a JSON which is then parsed to query the corresponding index. 

- Another one is Pydantic Selector which has a function calling APIs of LLMs. 


In [19]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [20]:
# let's check a few basic questions.

In [21]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: The question asks for a summary of the document, which aligns with the first choice about summarization..
The document provides a summary of financial data from three different files: Expenses.csv, Income.csv, and Savings.csv. The Expenses.csv file details various types of expenses for the months of March and April 2024, including categories like eating out, entertainment, rent, and utilities, among others. The Income.csv file outlines the income received by two individuals, Adam and Eve, in the same months, from sources such as salary, dividends, and rental income. Lastly, the Savings.csv file shows the amount saved by Adam and Eve in March and April, with details about the purpose of the savings, such as a percentage of salary, common savings, or specific goals like a home deposit or a car.


<span style="background-color: yellow;">Observe how it also displays reasoning behind choosing summary index to answer this query.</span>

In [22]:
response = query_engine.query("Compare expenses of April and March")
print(str(response))

Selecting query engine 1: The question requires retrieving specific context from the expenses tables for the months of April and March..
In April, the expenses were as follows: Eating Out - $182.00, Entertainment - $75.00, Government Process related fees - $1,898.00, Health - $288.00, Payment Plans and Loan Repayments - $541.00, Rent - $2,457.00, Shopping - $443.00, Subscriptions - $56.00, Transport - $305.00, Travel - $446.00, Utilities - $333.00, Groceries - $389.00, and Other - $191.00.

In March, the expenses were: Eating Out - $287.00, Entertainment - $219.00, Government Process related fees - $2,204.00, Health - $242.00, Payment Plans and Loan Repayments - $436.00, Rent - $3,393.00, Shopping - $196.00, Subscriptions - $48.00, Transport - $347.00, Travel - $124.00, Utilities - $200.00, Groceries - $241.00, and Other - $214.00. 

Comparing the two months, it appears that the expenses for most categories were higher in March than in April, with the exception of Health, Payment Plans

In [23]:
response = query_engine.query("what is the amount spent on groceries in the month of April")
print(str(response))

Selecting query engine 1: The question is asking for a specific piece of information (amount spent on groceries in April), which aligns with the description of choice 2 - retrieving specific context from incomes, savings and expenses tables..
The amount spent on groceries in the month of April is $389.00.


In [25]:
response = query_engine.query("How much do we have left from both our incomes after removing expenses for the month of April.")
print(str(response))

Selecting query engine 0: The question is asking for a summary of incomes and expenses for a specific month, which aligns with the first choice about summarization questions related to incomes, savings and expenses..
To calculate the remaining amount from both incomes after removing expenses for the month of April, we first need to sum up the total income and total expenses for that month.

The total income for April is the sum of the following amounts: 
- Adam's salary: $4049
- Adam's dividends: $9510
- Eve's salary: $3564
- Eve's dividends: $2000
- Adam's rental income: $2195
- Eve's rental income: $2841

This gives a total income of $24159.

The total expenses for April are the sum of the following amounts:
- Eating Out: $182
- Entertainment: $75
- Government Process related fees: $1898
- Health: $288
- Payment Plans and Loan Repayments: $541
- Rent: $2457
- Shopping: $443
- Subscriptions: $56
- Transport: $305
- Travel: $446
- Utilities: $333
- Other: $191
- Groceries: $389

This g

<span style="background-color: yellow;">Observe how the numbers were extracted correctly but the sum is incorrect.</span>

### Observations and Takeaways

- The router correctly selects the summary tool when the query requires an answer considering all chunks from our documents.
- It selects the vector tool when only a specific context within our documents is required to answer the query.
- While it fetches correct information, it produces a sum of expenses incorrectly. This can be handled by defining a tool.

Next Features:
- Design and implement tools to handle more complex queries. 